### Restart and Run All

In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3')
conlt = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)
pd.set_option('display.max_rows',None)

year = 2022
quarter = 1

today = date.today()
today

datetime.date(2022, 8, 15)

In [2]:
sql = '''
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s)
OR (year = %s-1 AND quarter >= %s+1) 
ORDER BY year DESC, quarter DESC'''
sql = sql % (year, quarter, year, quarter)
dfc = pd.read_sql(sql, conlt)
dfc['Counter'] = 1
dfc_grp = dfc.groupby(['name'], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp['Counter'] == 4]
dfc_grp

,name,year,quarter,q_amt,Counter
0,ACE,8085,10,1468062,4
1,ADVANC,8085,10,26589096,4
2,AEONTS,8085,10,3518454,4
3,AH,8085,10,1015701,4
4,AIE,8085,10,432491,4
5,AIMIRT,8085,10,524978,4
6,AIT,8085,10,571977,4
7,AJ,8085,10,467710,4
8,AMATA,8085,10,1770024,4
9,ANAN,8085,10,-744140,4


In [3]:
sql = '''
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s-1) 
OR (year = %s-1 AND quarter >= %s) 
ORDER BY year DESC, quarter DESC'''
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2022 AND quarter <= 1-1) 
OR (year = 2022-1 AND quarter >= 1) 
ORDER BY year DESC, quarter DESC


In [4]:
dfp = pd.read_sql(sql, conlt)
dfp['Counter'] = 1
dfp_grp = dfp.groupby(['name'], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp['Counter'] == 4]
dfp_grp.shape

(229, 5)

In [5]:
dfm = pd.merge(dfc_grp, dfp_grp, on='name', suffixes=(['_c','_p']), how='inner')
dfm['inc_profit'] = dfm['q_amt_c']-dfm['q_amt_p']
dfm['Pct'] = round(dfm['inc_profit']/abs(dfm['q_amt_p'])*100,2)
dfm['year'] = year
dfm['quarter'] = 'Q'+str(quarter)
df_pct = dfm[['name','year','quarter','q_amt_c','q_amt_p','inc_profit','Pct']]
df_pct

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
0,ACE,2022,Q1,1468062,1505681,-37619,-2.50
1,ADVANC,2022,Q1,26589096,26922146,-333050,-1.24
2,AEONTS,2022,Q1,3518454,3553041,-34587,-0.97
3,AH,2022,Q1,1015701,1023968,-8267,-0.81
4,AIE,2022,Q1,432491,423622,8869,2.09
5,AIMIRT,2022,Q1,524978,487775,37203,7.63
6,AIT,2022,Q1,571977,527125,44852,8.51
7,AJ,2022,Q1,467710,428622,39088,9.12
8,AMATA,2022,Q1,1770024,1402287,367737,26.22
9,ANAN,2022,Q1,-744140,-457342,-286798,-62.71


In [6]:
sql = """
DELETE FROM qt_profits 
WHERE year = %s AND quarter = 'Q%s'"""
sql = sql % (year,quarter)
print(sql)


DELETE FROM qt_profits 
WHERE year = 2022 AND quarter = 'Q1'


In [7]:
rp = conlt.execute(sql)
rp.rowcount

219

In [8]:
sql = 'SELECT name, id FROM tickers'
tickers = pd.read_sql(sql, conlt)
tickers.shape

(402, 2)

In [9]:
df_ins = pd.merge(df_pct, tickers, on='name', how='inner')
df_ins.dtypes

name           object
year            int64
quarter        object
q_amt_c         int64
q_amt_p         int64
inc_profit      int64
Pct           float64
id              int64
dtype: object

In [10]:
rcds = df_ins.values.tolist()
len(rcds)

225

In [11]:
for rcd in rcds:
    print(rcd)

['ACE', 2022, 'Q1', 1468062, 1505681, -37619, -2.5, 698]
['ADVANC', 2022, 'Q1', 26589096, 26922146, -333050, -1.24, 6]
['AEONTS', 2022, 'Q1', 3518454, 3553041, -34587, -0.97, 7]
['AH', 2022, 'Q1', 1015701, 1023968, -8267, -0.81, 9]
['AIE', 2022, 'Q1', 432491, 423622, 8869, 2.09, 720]
['AIMIRT', 2022, 'Q1', 524978, 487775, 37203, 7.63, 669]
['AIT', 2022, 'Q1', 571977, 527125, 44852, 8.51, 11]
['AJ', 2022, 'Q1', 467710, 428622, 39088, 9.12, 12]
['AMATA', 2022, 'Q1', 1770024, 1402287, 367737, 26.22, 21]
['ANAN', 2022, 'Q1', -744140, -457342, -286798, -62.71, 23]
['AOT', 2022, 'Q1', -17151697, -16322014, -829683, -5.08, 24]
['AP', 2022, 'Q1', 4870212, 4543060, 327152, 7.2, 25]
['ASIAN', 2022, 'Q1', 1077825, 1044316, 33509, 3.21, 36]
['ASK', 2022, 'Q1', 1277644, 1202804, 74840, 6.22, 38]
['ASP', 2022, 'Q1', 775000, 978355, -203355, -20.79, 40]
['ASW', 2022, 'Q1', 856966, 951364, -94398, -9.92, 728]
['AWC', 2022, 'Q1', 2100590, 861479, 1239111, 143.84, 699]
['BA', 2022, 'Q1', -8824265, -8549

In [12]:
sql = """
INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) \
 VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)


INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id)  VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [13]:
for rcd in rcds:
    conlt.execute(sql, rcd)

### End of loop

In [14]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit Pct'.split()

In [15]:
criteria_1 = df_ins.q_amt_c > 440000
df_ins.loc[criteria_1,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
54,CRC,2022,Q1,861862,59370,802492,"1,351.68"
73,GFPT,2022,Q1,604464,209344,395120,188.74
16,AWC,2022,Q1,2100590,861479,1239111,143.84
74,GGC,2022,Q1,722960,330219,392741,118.93
19,BANPU,2022,Q1,18580896,9851795,8729101,88.60


In [16]:
criteria_2 = df_ins.q_amt_p > 400000
df_ins.loc[criteria_2,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
16,AWC,2022,Q1,2100590,861479,1239111,143.84
19,BANPU,2022,Q1,18580896,9851795,8729101,88.60
66,EGCO,2022,Q1,7634185,4103840,3530345,86.03
68,ESSO,2022,Q1,7554525,4443214,3111311,70.02
171,SPRC,2022,Q1,8025156,4746462,3278694,69.08


In [17]:
criteria_3 = df_ins.Pct > 10
df_ins.loc[criteria_3,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
54,CRC,2022,Q1,861862,59370,802492,"1,351.68"
65,EGATIF,2022,Q1,388220,-123261,511481,414.96
73,GFPT,2022,Q1,604464,209344,395120,188.74
16,AWC,2022,Q1,2100590,861479,1239111,143.84
114,M,2022,Q1,313003,130979,182024,138.97


In [18]:
df_ins_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[df_ins_criteria,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
16,AWC,2022,Q1,2100590,861479,1239111,143.84
19,BANPU,2022,Q1,18580896,9851795,8729101,88.60
66,EGCO,2022,Q1,7634185,4103840,3530345,86.03
68,ESSO,2022,Q1,7554525,4443214,3111311,70.02
171,SPRC,2022,Q1,8025156,4746462,3278694,69.08
